## scores = 0.81075
sources
#### https://youtu.be/5nYqK-HaoKY
---------------
#### Credits to the Medium article
https://medium.com/optuna/lightgbm-tuner-new-optuna-integration-for-hyperparameter-optimization-8b7095e99258

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
df = df.drop(columns = 'id')
df_test = df_test.drop(columns = 'id')

In [ ]:
x = df.drop(columns = 'claim')
y = df.claim

In [ ]:
# import optuna
# from sklearn.model_selection import train_test_split
# import lightgbm as lgb
# from sklearn import metrics

# def objective(trial):
    
#     train_x, test_x, train_y, test_y = train_test_split(x,y, test_size=0.2)
 
#     params = {
#         'objective': 'binary',
#         'metric': 'auc',
#         'n_estimators': trial.suggest_categorical("n_estimators", [10000]),
#         'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.3),
#         'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
#         'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
#         'num_leaves': trial.suggest_int('num_leaves', 2, 100),
#         'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1.0),
#         'bagging_fraction': trial.suggest_float("bagging_fraction", 0.2, 0.95, step=0.1),
#         'subsample': trial.suggest_float("subsample", 0.2, 0.9, step=0.1),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#     }
    
#     model = lgb.LGBMClassifier(**params,device = 'gpu')
#     model.fit(train_x,train_y,eval_set=[(test_x,test_y)], early_stopping_rounds=50)

#     preds = model.predict_proba(test_x)[:,1]
#     roc_auc_score = metrics.roc_auc_score(test_y,preds)
    
#     return roc_auc_score


# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=50)
 
# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

In [ ]:
# Number of finished trials: 50
# Best trial: {'n_estimators': 10000, 'learning_rate': 0.039373014449069724, 'lambda_l1': 1.3580250554531046, 'lambda_l2': 7.420227454832275e-05, 'num_leaves': 84, 'feature_fraction': 0.7675264126841835, 'bagging_fraction': 0.7, 'subsample': 0.4, 'min_child_samples': 24}

In [ ]:
##create folds
from sklearn import model_selection
df["kfold"] = -1

df = df.sample(frac=1).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=5, shuffle=False, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X=df, y=df.claim.values)):
    print(len(train_idx), len(val_idx))
    df.loc[val_idx, 'kfold'] = fold

## Ensemble

### Train.py

In [ ]:
params ={'device':['gpu'],'n_estimators': 10000, 'learning_rate': 0.039373014449069724, 
         'lambda_l1': 1.3580250554531046, 'lambda_l2': 7.420227454832275e-05, 
         'num_leaves': 84, 'feature_fraction': 0.7675264126841835, 
         'bagging_fraction': 0.7, 'subsample': 0.4, 'min_child_samples': 24}

In [ ]:
import joblib
import lightgbm as lgb
import xgboost as xg
import pandas as pd
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
def run(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    df_train = df_train.drop(columns = 'kfold')
    df_valid = df_valid.drop(columns = 'kfold')
    x_train = df_train.drop('claim', axis=1).values
    y_train = df_train.claim.values
    x_valid = df_valid.drop('claim', axis=1).values
    y_valid = df_valid.claim.values
    clf = lgb.LGBMClassifierclf = lgb.LGBMClassifier(device = 'gpu',
                            learning_rate= 0.039373014449069724,
                            n_estimators= 1000,
                            lambda_l1=  1.3580250554531046, lambda_l2 =  7.420227454832275e-05,subsample= 0.4,
                            num_leaves= 84, feature_fraction= 0.7675264126841835,
                            bagging_fraction=  0.7, min_child_samples= 24)
    clf.fit(x_train, y_train)
    y_pred = clf.predict_proba(x_valid)[:,1]
    roc_auc_score = metrics.roc_auc_score(y_valid,y_pred)
    print(f"Fold={fold}, roc_auc_score={roc_auc_score}")
    File_name = 'model_lgb' + str(fold)
    joblib.dump(
    clf,File_name)
for i in range(5):
    run(fold = i)

In [ ]:
model_0_lgb= joblib.load('./model_lgb0')
model_1_lgb =joblib.load('./model_lgb1')
model_2_lgb= joblib.load('./model_lgb2')
model_3_lgb= joblib.load('./model_lgb3')
model_4_lgb= joblib.load('./model_lgb4')

In [ ]:
y_final_3_lgb = model_3_lgb.predict_proba(df_test)[:,1]
y_final_0_lgb = model_0_lgb.predict_proba(df_test)[:,1]
y_final_1_lgb = model_1_lgb.predict_proba(df_test)[:,1]
y_final_2_lgb = model_2_lgb.predict_proba(df_test)[:,1]
y_final_4_lgb = model_4_lgb.predict_proba(df_test)[:,1]

In [ ]:
y_final_avg = (y_final_0_lgb + y_final_1_lgb +y_final_2_lgb + y_final_3_lgb + y_final_4_lgb)/5

In [ ]:
submission['claim'] = y_final_avg 

In [ ]:
submission.to_csv('pred_csv_lgb_avg.csv',index = False)